Install dependencies with:
```sh
pip install -r requirements.txt
```

For some functionality you need a valid API-key.  
Export as environment variables.
Afterwards run notebook from same shell.


Bash/Sh/Zsh:
```sh
export BINANCE_API_KEY='your_api_key'
export BINANCE_API_SECRET='your_secret_key'

# Check
echo $BINANCE_API_KEY
echo $BINANCE_API_SECRET
```
Powershell:
```powershell
$env:BINANCE_API_KEY = 'your_api_key'
$env:BINANCE_API_SECRET = 'your_secret_key'

# Check
$env:BINANCE_API_KEY
$env:BINANCE_API_KEY
```

In [67]:
import os
import pandas as pd
import numpy as np
from binance.spot import Spot as Client
pd.set_option('display.max_rows', 80)
pd.options.display.float_format = '{:.3f}'.format

In [2]:
# Ommit base_url argument to default to api.binance.com
client = Client(os.getenv('BINANCE_API_KEY'), os.getenv('BINANCE_API_SECRET'), base_url='https://testnet.binance.vision')
print(client.time())

{'serverTime': 1652032595202}


In [3]:
pd.json_normalize(client.account()['balances'])

,asset,free,locked
0,BNB,1000.00000000,0.00000000
1,BTC,1.00000000,0.00000000
2,BUSD,10000.00000000,0.00000000
3,ETH,100.00000000,0.00000000
4,LTC,500.00000000,0.00000000
5,TRX,500000.00000000,0.00000000
6,USDT,10000.00000000,0.00000000
7,XRP,50000.00000000,0.00000000


In [4]:
symbols = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'LTCUSDT', 'TRXUSDT', 'XRPUSDT' ]
columns =  ['Open time','Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore' ]

In [23]:
assets = pd.concat(([pd.DataFrame(client.klines(symbol, "1d"), columns=columns) for symbol in symbols]), axis = 1, keys=symbols)
assets

BTCUSDT                                                   \
       Open time            Open             High             Low   
0  1651622400000  38127.16000000   67952.01000000   9000.00000000   
1  1651708800000  39690.01000000   48256.09000000  12000.00000000   
2  1651795200000  36552.97000000   90000.00000000   9500.00000000   
3  1651881600000  35851.70000000  179685.00000000   2000.00000000   
4  1651968000000  35472.40000000  172120.00000000   6988.00000000   

                                                                     \
            Close         Volume     Close Time  Quote asset volume   
0  39695.80000000  3289.47888900  1651708799999  128626868.06394152   
1  36551.04000000  3747.85911700  1651795199999  142179013.42520536   
2  35844.54000000  5133.87878000  1651881599999  185618082.67083564   
3  35472.39000000  2648.74837000  1651967999999   94693433.70235498   
4  33993.38000000  4490.48023700  1652054399999  155345040.35428542   

                                                ...     XRPUSDT              \
  Number of trades Taker buy base asset volume  ...        High         Low   
0           109403               2070.21409400  ...  0.64990000  0.60830000   
1           136426               2305.80703300  ...  0.65710000  0.58040000   
2           195320               2791.12783400  ...  0.61630000  0.58130000   
3           110202               1439.33242400  ...  0.60340000  0.56940000   
4           164365               2561.20209400  ...  0.58280000  0.55810000   

                                                                    \
        Close             Volume     Close Time Quote asset volume   
0  0.64610000   7188003.90000000  1651708799999   4497422.00637000   
1  0.59830000  11682556.70000000  1651795199999   7147202.83414000   
2  0.58130000  12824794.10000000  1651881599999   7706389.17155000   
3  0.58180000   4378006.60000000  1651967999999   2579982.36840000   
4  0.56670000   7913078.00000000  1652054399999   4526310.99851000   

                                                                             \
  Number of trades Taker buy base asset volume Taker buy quote asset volume   
0             8638            7020708.90000000             4393175.81487000   
1            13976           11566933.90000000             7074742.84037000   
2            15203           12610238.80000000             7578080.05335000   
3             5475            4097615.10000000             2413970.94234000   
4             8610            7662390.60000000             4382803.76378000   

          
  Ignore  
0      0  
1      0  
2      0  
3      0  
4      0  

[5 rows x 72 columns]

Swapping levels for easier selection 

In [49]:
assets = assets.swaplevel(axis=1)
assets

,Open time,Open,High,Low,Close,Volume,Close Time,Quote asset volume,Number of trades,Taker buy base asset volume,...,High,Low,Close,Volume,Close Time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
,BTCUSDT,BTCUSDT,BTCUSDT,BTCUSDT,BTCUSDT,BTCUSDT,BTCUSDT,BTCUSDT,BTCUSDT,BTCUSDT,...,XRPUSDT,XRPUSDT,XRPUSDT,XRPUSDT,XRPUSDT,XRPUSDT,XRPUSDT,XRPUSDT,XRPUSDT,XRPUSDT
0,1651622400000,38127.16000000,67952.01000000,9000.00000000,39695.80000000,3289.47888900,1651708799999,128626868.06394152,109403,2070.21409400,...,0.64990000,0.60830000,0.64610000,7188003.90000000,1651708799999,4497422.00637000,8638,7020708.90000000,4393175.81487000,0
1,1651708800000,39690.01000000,48256.09000000,12000.00000000,36551.04000000,3747.85911700,1651795199999,142179013.42520536,136426,2305.80703300,...,0.65710000,0.58040000,0.59830000,11682556.70000000,1651795199999,7147202.83414000,13976,11566933.90000000,7074742.84037000,0
2,1651795200000,36552.97000000,90000.00000000,9500.00000000,35844.54000000,5133.87878000,1651881599999,185618082.67083564,195320,2791.12783400,...,0.61630000,0.58130000,0.58130000,12824794.10000000,1651881599999,7706389.17155000,15203,12610238.80000000,7578080.05335000,0
3,1651881600000,35851.70000000,179685.00000000,2000.00000000,35472.39000000,2648.74837000,1651967999999,94693433.70235498,110202,1439.33242400,...,0.60340000,0.56940000,0.58180000,4378006.60000000,1651967999999,2579982.36840000,5475,4097615.10000000,2413970.94234000,0
4,1651968000000,35472.40000000,172120.00000000,6988.00000000,33993.38000000,4490.48023700,1652054399999,155345040.35428542,164365,2561.20209400,...,0.58280000,0.55810000,0.56670000,7913078.00000000,1652054399999,4526310.99851000,8610,7662390.60000000,4382803.76378000,0


Formula to convert from unix time to datetime, needs to be set as index

In [6]:
pd.concat([pd.to_datetime(assets[(symbol, 'Open time')], unit= 'ms') for symbol in symbols], axis=1)

,BTCUSDT,ETHUSDT,BNBUSDT,LTCUSDT,TRXUSDT,XRPUSDT
,Open time,Open time,Open time,Open time,Open time,Open time
0,2022-05-04,2022-05-04,2022-05-04,2022-05-04,2022-05-04,2022-05-04
1,2022-05-05,2022-05-05,2022-05-05,2022-05-05,2022-05-05,2022-05-05
2,2022-05-06,2022-05-06,2022-05-06,2022-05-06,2022-05-06,2022-05-06
3,2022-05-07,2022-05-07,2022-05-07,2022-05-07,2022-05-07,2022-05-07
4,2022-05-08,2022-05-08,2022-05-08,2022-05-08,2022-05-08,2022-05-08


Converting string objects to float

In [ ]:
close = assets["Close"].copy().astype(float)

Returns

In [71]:
returns = close.pct_change().dropna()

In [70]:
returns.head()

,BTCUSDT,ETHUSDT,BNBUSDT,LTCUSDT,TRXUSDT,XRPUSDT
1,-0.079,-0.066,-0.060,-0.089,-0.136,-0.074
2,-0.019,-0.034,-0.585,-0.013,0.096,-0.028
3,-0.010,-0.007,1.327,-0.012,0.022,0.001
4,-0.042,-0.043,-0.032,-0.015,0.043,-0.026
